# Lab 3.1 - Function calling

Note: You can also follow the instructions here https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/function-calling?tabs=non-streaming%2Cpython

In [ ]:
# Step 1 - Load the environment variables with dotenv
from dotenv import load_dotenv
load_dotenv()
import os

# Endpoint and API key can be found in Azure AI Studio -> Project Settings -> Project Properties -> Get API endpoints
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
# The name of the deployment to tbe used. Found in Azure AI Studio -> Deployments
AZURE_OPENAI_DEPLOYMENT_ID = os.getenv("AZURE_OPENAI_DEPLOYMENT_ID")

In [ ]:
# Step 2 - Import libraries and create client
import os
from openai import AzureOpenAI
import json

client = AzureOpenAI(
  azure_endpoint = AZURE_OPENAI_ENDPOINT, 
  api_key=AZURE_OPENAI_API_KEY,  
  api_version="2024-03-01-preview"
)

In [ ]:
# Step 3 - Ask about the weather in a city
messages = [
    {
        "role": "user", 
        "content": "What's the weather like in San Francisco, Tokyo, and Paris?"
    }]
response = client.chat.completions.create(
    model=AZURE_OPENAI_DEPLOYMENT_ID,
    messages=messages
)
response_message = response.choices[0].message.content

print(response_message)

In [ ]:
# Step 4 - Define a function to be called

# Example function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

In [ ]:
# Step 5 - pass the get_current_weather function as a tool to the OpenAI model
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

# Notice the tools and tool_choice paramenters
response = client.chat.completions.create(
    model=AZURE_OPENAI_DEPLOYMENT_ID,
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)

response_message = response.choices[0].message
tool_calls = response_message.tool_calls

In [ ]:
# Step 6 - check if the model wanted to call a function
if tool_calls:
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        # only one function in this example, but you can have multiple
        "get_current_weather": get_current_weather,
    }  
    # extend conversation with assistant's reply
    messages.append(response_message)  
    
    # Step 4: send the info for each function call and function response to the model
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )  
    # extend conversation with function response
    second_response = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT_ID,
        messages=messages,
    )  # get a new response from the model where it can see the function response
    print(second_response.choices[0].message.content)
